In the previous notebook, we implemented a simple Perceptron algorithm and explained all the important parts of it. We concluded with a statement that, Perceptron has its own limitation which caused the winter of ANNs in the 1970s.   

In short, the limitation is that the Perceptron algorithm only works on [linearly separable](https://en.wikipedia.org/wiki/Linear_separability) problems. For example, in the following figure, we have two cases. In each case, we have two classes with blue circles, and red triangles. The perceptron can solve the left problem without any issues (you can see the green line can separate the two classes very well). But on the right side, we can see that there is no single line that can be drawn to separate the two classes, hence, the perceptron algorithm fails to classify them.   

<img src="https://raw.githubusercontent.com/qingkaikong/blog/master/40_ANN_part3_step_by_step_MLP/figures/figure1_Perceptron_limitation.jpg" width="600"/> 

You may have already guessed that the solution to the above problem is today's topic - the Multi-Layer Perceptron (MLP), a very popular Artificial Neural Network algorithm that you will use it a lot. It turns out, to solve the above problem, all we need is adding one more layer between the input and output layer. As we talked before, this is the hidden layer, which can actually capture the non-linear relationship in the data, thus can solve the problem by drawing a non-linear boundary to classify the data. Of course, you can add more layers in between, but we will just add one layer for simplicity. (The field of adding more layers to model more combinations of relationships such as this is known as ["deep learning"](https://en.wikipedia.org/wiki/Deep_learning) because of the increasingly deep layers being modeled.)

Let's use the same example we used last time, 5 training data samples, and each has 3 features as shown in the following table.   

|Feature1|Feature2|Feature3|Target|
|:------:|:------:|:------:|:----:|
|    0   |    0   |    1   |   0  |
|    1   |    1   |    1   |   1  |
|    1   |    0   |    1   |   1  |
|    0   |    1   |    1   |   0  |
|    0   |    1   |    0   |   1  |

But this time, we will build a MLP model to solve the same problem, i.e. we will add one hidden layer. In this newly added hidden layer, we have 4 nodes (neurons) which can be represented as the following:  

<img src="https://raw.githubusercontent.com/qingkaikong/blog/master/40_ANN_part3_step_by_step_MLP/figures/figure2_MLP_structure.jpg" width="600"/>  

We can see the structure of this Multilayer perceptron is more complicated than the simple perceptron. It has 3 layers
1. An input layer
2. A hidden layer
3. An output layer

The hidden layer has 4 nodes (neurons). The summation sign $\sum$ and activation function $f$ in the neurons of the hidden layer indicate that each neuron will work as before: take the weighted sum of the input data and the weights, and pass this summation through our sigmoid activation function. The generated values from these hidden neurons are the outputs for the hidden layer, which in turn become the inputs for the output layer. Between each layer, we added the bias term to avoid the 0 input problem we described in the previous notebook. Overall, there are several more weights in the model. Each arrow in the figure is one weight, and we can see that, for each node in one layer, it will connect to all the nodes in the next layer. 

Since most of the important parts of the MLP are similar to the Perceptron model we discussed in the previous notebook, we only go over the major differences here. 

### Forward Propogation
As before, the inputs propagate forward through the network to the output layer. We can think of each neuron in the hidden layer as a simple perceptron we talked before, with 4 copies of a parallel perceptron that not interact with each other, each with their own weights and biases as the following figure shows (note that I highlight the part that is essentially a perceptron structure). 

*note - unify language to say either neuron or node*

<img src="https://raw.githubusercontent.com/qingkaikong/blog/master/40_ANN_part3_step_by_step_MLP/figures/figure3_input_perceptron.jpg" width="600"/> 

After the neurons get the data form the input layers, they take the weighted sum and pass that to the sigmoid function which outputs a value between 0-1. The outputs from the hidden layer will be the input for the output layer, a strcuture analogous to the simple perceptron we previously built.

<img src="https://raw.githubusercontent.com/qingkaikong/blog/master/40_ANN_part3_step_by_step_MLP/figures/figure4_hidden_perceptron.jpg" width="600"/> 

### Backprogation - Learning

The learning or training of the MLP is more complicated, since now we have three layer with two sets of weights that need to be updated each time from the error. The method that we are going to use is called 'back-propagation', which makes it clear that the errors are sent backwards through the network. The best way to describe back-propagation properly is mathematically, but that is beyond the scope of this workshop. Instead, the purpose of this notebook is to show you a high level overview of how it works. If you want to learn more, check out this blog - [a step by step backpropagation example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/).  

Let's look at the code to implement a simple version of MLP, and explain the lines that we didn't cover in the previous notebook.  

In [9]:
import numpy as np

# define the sigmoid function
def sigmoid(x,deriv=False):
    sig = 1/(1+np.exp(-x))
    if(deriv==True):
        return sig*(1-sig)
    return sig

# define learning rate
learning_rate = 0.4

# the input data, and we add the bias in line 14
X = np.array([[0,0,1],
              [0,1,1],
              [1,0,1],
              [1,1,1],
              [0,1,0]])
X = np.concatenate((np.ones((len(X), 1)), X), axis = 1)

y = np.array([[0],[1],[1],[0],[1]])

np.random.seed(1)

# randomly initialize our weights with mean 0 for weights 
# connect input layer and hidden layer, and connect hidden
# layer to output layer
weights_0 = 2*np.random.random((4,4)) - 1
weights_1 = 2*np.random.random((5,1)) - 1

# training the model for 60000 iterations
for j in range(60000):

    # Feed forward through layers 0, 1, and 2
    # input layer
    layer_0 = X
    # layer_1_output is the output from the hidden layer
    layer_1_output = sigmoid(np.dot(layer_0,weights_0))
    # Note here we add a bias term before we feed them into the output layer
    layer_1_output = np.concatenate((np.ones((len(layer_1_output), 1)), layer_1_output), axis = 1)
    
    # layer_2_output is the estimation the model made using current weights
    layer_2_output = sigmoid(np.dot(layer_1_output,weights_1))

    # how much did we miss the target value?
    layer2_error = y - layer_2_output
    
    # let's print out the error we made at each 10000 iteration
    if (j% 10000) == 0:
        print("Error:" + str(np.mean(np.abs(layer2_error)).round(5)))

    # How much we will change for the weights connect hidden layer
    # and output layer
    layer2_delta = learning_rate*layer2_error*sigmoid(layer_2_output,deriv=True)

    # how much did each hidden node value contribute to the output error (according to the weights)?
    layer1_error = layer2_delta.dot(weights_1.T)

    # How much we will change for the weights connect the input layer
    # and the hidden layer
    layer1_delta = learning_rate*layer1_error * sigmoid(layer_1_output,deriv=True)

    # update the weights
    weights_1 += layer_1_output.T.dot(layer2_delta)
    weights_0 += layer_0.T.dot(layer1_delta[:, 1:])
    
print("Output After Training:")
print(layer_2_output.round(2))
print('Target')
print(y)

Error:0.52021
Error:0.02142
Error:0.00953
Error:0.00641
Error:0.00486
Error:0.00392
Output After Training:
[[0.]
 [1.]
 [1.]
 [0.]
 [1.]]
Target
[[0]
 [1]
 [1]
 [0]
 [1]]


## Explain line by line  

Most of the lines are very similar to the previous perceptron model with the exception of lines 44 and 52.

**Line 44:** we print the error at each 10,000th iteration (i.e. 10,000 , 20,000, ... , 60,000) with the expectation that the error will decrease as we update the weights. The more iterations we have, the smaller the error will be.  

```python
    # let's print out the error we made at each 10000 iteration
    if (j% 10000) == 0:
        print("Error:" + str(np.mean(np.abs(layer2_error))))
```

**Line 52:** uses the "confidence weighted error" from the output layer to establish an error for the hidden layer. We simply sends the error across the weights from output layer to hidden layer. This gives what you might call a "contribution weighted error" because we learn how much each node value in the hidden layer "contributes" to the error in output layer. We then update the weights that connect the input layer to the hidden layer using the same steps we did in the perceptron implementation.   

```python
    # how much did each hidden node value contribute to the output error (according to the weights)?
    layer1_error = layer2_delta.dot(weights_1.T)
```


## Visualize ANN  
Hopefully you have a better understanding of the ANN algorithm after reading these notebooks, which cover the most important concepts of the ANN! Luckily, we don't need to implement ANN from scratch, since there are already many packages out there for us to use. Next notebook, we will try to use the implementation of the [MLP in sklearn](http://scikit-learn.org/stable/modules/neural_networks_supervised.html) to classify the handwriting digits, while learn more about the Multi-Layer Perceptron that we haven't covered yet. 

## References and Acknowledgments

[A Neural Network in 11 lines of Python](http://iamtrask.github.io/2015/07/12/basic-python-network/) (I thank the author, since my example is modified from his blog).    
[Machine learning - An Algorithmic Perspective](https://seat.massey.ac.nz/personal/s.r.marsland/MLBook.html)   
[Single-Layer Neural Networks and Gradient Descent](http://sebastianraschka.com/Articles/2015_singlelayer_neurons.html)